In [1]:
import pandas as pd
import re
import math
import numpy as np

# Data Preprocessing

In [2]:
df_orig = pd.read_excel("masoud.xls", skiprows=10, skipfooter=4)

WARNING *** file size (140071) not 512 + multiple of sector size (512)


In [3]:
df_orig.head()

,Unnamed: 0,Data Negócio,Unnamed: 2,C/V,Mercado,Prazo,Código,Especificação do Ativo,Quantidade,Preço (R$),Valor Total (R$)
0,NaN,12/03/20,NaN,C,Mercado a Vista,NaN,COGN3,COGNA ON ON NM,100,6.40,640.00
1,NaN,12/03/20,NaN,C,Mercado a Vista,NaN,PETR4,PETROBRAS PN N2,100,12.65,1265.00
2,NaN,12/03/20,NaN,C,Merc. Fracionário,NaN,BPAC11F,BTGP BANCO UNT N2,30,39.00,1170.00
3,NaN,12/03/20,NaN,C,Merc. Fracionário,NaN,MGLU3F,MAGAZ LUIZA ON NM,35,33.85,1184.75
4,NaN,12/03/20,NaN,C,Merc. Fracionário,NaN,PRIO3F,PETRORIO ON NM,38,17.57,667.66


In [4]:
df_orig.drop([x for x in df_orig.columns if x.startswith("Unn")], axis=1 , inplace=True)

In [5]:
df_orig['Data Negócio'] = pd.to_datetime(df_orig['Data Negócio'],dayfirst=True)
#df_orig['Data Negócio'] = df_orig['Data Negócio'].dt.date

Correcting the problem in the column "C/V

In [6]:
def correction(x):
    if "C" in x:
        x = "C"
    elif 'V' in x:
        x = "V"
    return x


df_orig['C/V']=df_orig["C/V"].apply(correction)

In [7]:
def fracionario_to_normal(x):
    if x.endswith("F"):
        x = x[:-1]
    return x

df_orig['Código'] = df_orig["Código"].apply(fracionario_to_normal)


In [8]:
df_orig.drop(['Mercado', 'Prazo',
       'Especificação do Ativo',], axis=1, inplace=True)

In [9]:
df_orig.head()

,Data Negócio,C/V,Código,Quantidade,Preço (R$),Valor Total (R$)
0,2020-03-12,C,COGN3,100,6.40,640.00
1,2020-03-12,C,PETR4,100,12.65,1265.00
2,2020-03-12,C,BPAC11,30,39.00,1170.00
3,2020-03-12,C,MGLU3,35,33.85,1184.75
4,2020-03-12,C,PRIO3,38,17.57,667.66


In [14]:
def check_consistency():

    df = pd.read_csv("df.csv", index_col=0)
    df['Data Negócio'] = pd.to_datetime(df['Data Negócio'],dayfirst=True)
    
    #status = True

    fail = {'ticker':'', 'date':'', "index":'' }
    for ticker in df["Código"].unique():

        if len(df[(df["C/V"] == "V") & (df["Código"] == ticker)].values) > 0:
	#check to insure the sell date is after the purchase date. It guarantee that we have a mean price to calculate the profit.
            
            first_sell_index = df[(df["C/V"] == "V") & (df["Código"] == ticker)]['Data Negócio'].index[0]
            first_sell_date = df.iloc[first_sell_index]['Data Negócio']

         
	#check if the sold ticker has a purchased price. It is important to calculate the mean price of the ticker and then profit.
            if len(df[(df["C/V"] == "C") & (df["Código"] == ticker)]['Data Negócio'].iloc[:first_sell_index]) == 0:
                fail['ticker'] = ticker
                fail['index'] = first_sell_index
                fail['date'] = first_sell_date
                status = True
                break
                
            else:
                status = False        
                
    return status ,fail

In [16]:
check_consistency()

(False, {'ticker': '', 'date': '', 'index': ''})

# General View

This table specify the day and swing trades. mean cost of the purchased stocks. profit of the sold stocks. 

In [16]:
df = df_orig.copy()

Finding the day-trade negotiations

In [271]:
day_trade = {"date":[], "ticker":[], 'index':[]}

dates = df["Data Negócio"].unique()

for date in dates:
    tickers = df[df["Data Negócio"]==date]['Código'].unique()
    for ticker in tickers:
        if all(x in df[(df["Data Negócio"]==date) & (df["Código"]==ticker)]["C/V"].values for x in ["C","V"]):
            day_trade['index'].append(df[(df["Data Negócio"]==date) & (df["Código"]==ticker)]["C/V"].index)
            day_trade['date'].append(date)
            day_trade['ticker'].append(ticker)

day_trade["index"] = [item for sublist in day_trade['index'] for item in sublist]

In [272]:
df_general = df.copy()

Creating the "day/swing" column

In [273]:
df_general['Day/Swing'] = "Swing"

df_general.at[day_trade['index'], 'Day/Swing'] = 'Day'

Calculating the mean cost for each purchase

In [274]:
df_general.head()

,Data Negócio,C/V,Código,Quantidade,Preço (R$),Valor Total (R$),Day/Swing
0,2020-05-08,C,COGN3,10000,4.70,47000.0,Swing
1,2020-05-08,C,TAEE4,100,8.90,890.0,Swing
2,2020-05-13,C,IRDM11,5,100.42,502.1,Swing
3,2020-05-13,C,WEGE3,3,38.00,114.0,Swing
4,2020-05-20,C,BBAS3,35,27.10,948.5,Swing


In [299]:
df_general["Valor Total (R$)"] = np.where(df_general["C/V"] == "C", -1* df_general["Valor Total (R$)"], df_general["Valor Total (R$)"])

#df_swing["Quantidade"] = np.where(df_swing["C/V"] == "V", -1* df_swing["Quantidade"], df_swing["Quantidade"])

df_general['Custo de Operação'] = np.where(df_general["C/V"] == "V", -1*df_general['Valor Total (R$)'] * (0.000325 + 0.00005),df_general['Valor Total (R$)'] * (0.000325))

#df_general['Custo de Operação'] = round(df_general['Custo de Operação'],6)

In [300]:
df_general.head()

,Data Negócio,C/V,Código,Quantidade,Preço (R$),Valor Total (R$),Day/Swing,Custo de Operação,Custo Médio,Lucro da Venda
0,2020-05-08,C,COGN3,10000,4.70,-47000.0,Swing,-15.275000,-4.702,0.0
1,2020-05-08,C,TAEE4,100,8.90,-890.0,Swing,-0.289250,-8.903,0.0
2,2020-05-13,C,IRDM11,5,100.42,-502.1,Swing,-0.163182,-100.453,0.0
3,2020-05-13,C,WEGE3,3,38.00,-114.0,Swing,-0.037050,-38.012,0.0
4,2020-05-20,C,BBAS3,35,27.10,-948.5,Swing,-0.308262,-27.109,0.0


In [301]:
tickers = df_general["Código"].unique() 
df_general["Custo Médio"] = 0.

for ticker in tickers:
    means = {"Custo":[], "N":[]}
    for index, row in df_general[(df_general["Código"] == ticker)&(df_general['C/V']=='C')].iterrows():
        means["Custo"].append(row["Valor Total (R$)"] +  row['Custo de Operação'])
        means["N"].append(row["Quantidade"])
        mean = -1*sum(means["Custo"])/(sum(means['N']))
        df_general.at[index,'Custo Médio'] = round(mean,3)




Calculating the profit for each sell

In [302]:
df_general["Lucro da Venda"] = 0.

indices = df_general[df_general["C/V"] == 'V'].index

for index in indices:
    quantity = df_general.loc[index]["Quantidade"]
    total = df_general.loc[index]["Valor Total (R$)"] + df_general.loc[index]["Custo de Operação"]
    ticker = df_general.loc[index]["Código"]
    custo_medio = df_general[(df_general["Código"] ==ticker) & (df_general["C/V"] == 'C')].loc[:index]["Custo Médio"].values[-1]
    df_general.at[index, "Lucro da Venda"] = round(total - (quantity * custo_medio),2)


## Final Script

In [54]:
def general_view(df1):

    df=df1.copy()

    #finding the day-trade operations
    day_trade = {"date":[], "ticker":[], 'index':[]}

    dates = df["Data Negócio"].unique()

    for date in dates:
        tickers = df[df["Data Negócio"]==date]['Código'].unique()
        for ticker in tickers:
            if all(x in df[(df["Data Negócio"]==date) & (df["Código"]==ticker)]["C/V"].values for x in ["C","V"]):
                day_trade['index'].append(df[(df["Data Negócio"]==date) & (df["Código"]==ticker)]["C/V"].index)
                day_trade['date'].append(date)
                day_trade['ticker'].append(ticker)

    day_trade["index"] = [item for sublist in day_trade['index'] for item in sublist]

    #creating a new column to mark the swing/day trades

    df['Day/Swing'] = "Swing"

    df.at[day_trade['index'], 'Day/Swing'] = 'Day'

    #Calculating the operational costs of the negotiations.

    df["Valor Total (R$)"] = np.where(df["C/V"] == "C", -1* df["Valor Total (R$)"], df["Valor Total (R$)"])


    df['Custo de Operação'] = np.where(df["C/V"] == "V", -1*df['Valor Total (R$)'] * (0.000325 + 0.00005),df['Valor Total (R$)'] * (0.000325))

    df['Custo de Operação'] = round(df['Custo de Operação'],3)

    
    #calculationg the mean cost of a purchased stock and its evolution by new acquisition for swing trade. 
    tickers = df["Código"].unique() 
    df["Custo Médio"] = 0.

    for ticker in tickers:
        means = {"Custo":[], "N":[]}
        for index, row in df[(df["Código"] == ticker) & (df['C/V']=='C')].iterrows():
            if index not in day_trade["index"]:
                means["Custo"].append(row["Valor Total (R$)"] +  row['Custo de Operação'])
                means["N"].append(row["Quantidade"])
                mean = -1*sum(means["Custo"])/(sum(means['N']))
                df.at[index,'Custo Médio'] = round(mean,3)
    
    #calculating custo medio for day-trade operations
    for date in day_trade["date"]:
        for ticker in df[df['Data Negócio']==date]["Código"].unique():
            if ticker in day_trade['ticker']:
                day_indices = df[(df['Data Negócio']==date) & (df['Código']==ticker) & (df["C/V"]=='C')].index
                total_quantity = df.iloc[day_indices]["Quantidade"].sum()
                price_sum = df.iloc[day_indices]["Valor Total (R$)"].sum()
                cost_sum = df.iloc[day_indices]['Custo de Operação'].sum()
                total = price_sum + cost_sum
                df.at[day_indices, 'Custo Médio'] = -1*round(total/total_quantity, 3)



    #calculating the profit of each sell

    df["Lucro da Venda"] = 0. 

    indices = df[df["C/V"] == 'V'].index

    for index in indices:
        if df.iloc[index]['Day/Swing'] == "Swing":
            quantity = df.iloc[index]["Quantidade"]
            total = df.iloc[index]["Valor Total (R$)"] + df.iloc[index]["Custo de Operação"]
            ticker = df.iloc[index]["Código"]
            custo_medio = df[(df["Código"] ==ticker) & (df["C/V"] == 'C')].iloc[:index]["Custo Médio"].values[-1]
            df.at[index, "Lucro da Venda"] = round(total - (quantity * custo_medio),3)

        if df.iloc[index]['Day/Swing'] == "Day":
            total = df.iloc[index]["Valor Total (R$)"] + df.iloc[index]["Custo de Operação"]
            ticker = df.iloc[index]["Código"]
            quantity = df.iloc[index]["Quantidade"]
            date = df.iloc[index]["Data Negócio"]
            custo_medio = df[(df["Código"] ==ticker) & (df["C/V"] == 'C')&(df["Data Negócio"]==date)]["Custo Médio"].values[0]
            df.at[index, "Lucro da Venda"] = round(total - (quantity * custo_medio),3)




    

    return df

In [55]:
test = general_view(df_orig)

In [56]:
test[test['Código'] == "OIBR4"]

,Data Negócio,C/V,Código,Quantidade,Preço (R$),Valor Total (R$),Day/Swing,Custo de Operação,Custo Médio,Lucro da Venda
23,2020-07-23,C,OIBR4,24,1.67,-40.08,Swing,-0.013,1.671,0.000
24,2020-07-23,C,OIBR4,11,1.67,-18.37,Swing,-0.006,1.671,0.000
25,2020-07-23,C,OIBR4,64,1.67,-106.88,Swing,-0.035,1.671,0.000
42,2020-07-29,C,OIBR4,300,4.08,-1224.00,Day,-0.398,4.081,0.000
46,2020-07-29,V,OIBR4,25,3.90,97.50,Day,-0.037,0.000,-4.562
47,2020-07-29,V,OIBR4,20,3.90,78.00,Day,-0.029,0.000,-3.649
48,2020-07-29,V,OIBR4,4,3.90,15.60,Day,-0.006,0.000,-0.730
49,2020-07-29,V,OIBR4,49,3.90,191.10,Day,-0.072,0.000,-8.941
50,2020-07-29,V,OIBR4,1,3.90,3.90,Day,-0.001,0.000,-0.182


The below function depends on the previsouly sliced dataframe to calculate the tax

In [11]:
def day_trade_imposto(df):

    df = df[df["Day/Swing"]=="Day"].copy()
    for index in df.index:
            df.at[index, "DARF"] = df.loc[index]["Lucro da Venda"]* 0.20
    
    df_group = df[['Data Negócio', 'C/V', 'Código', 'Quantidade', 
    'Valor Total (R$)', 'Custo de Operação', 'Lucro da Venda',  "Day/Swing", "DARF"]].groupby(['Data Negócio', "Código", "C/V"]).sum()

    imposto_day = df_group['DARF'].sum()

    
    print(f"O total imposto devido em relação as operações day-trade no periodo escolhido é {round(imposto_day,2)}")

    return df_group[df_group['DARF']!=0]

In [42]:
def swing_trade_imposto(df):

    df = df[df["Day/Swing"]=="Swing"].copy()
        
    for y in df["Data Negócio"].dt.year.unique():
        for m in df[df["Data Negócio"].dt.year ==y]["Data Negócio"].dt.month.unique():
            for ticker in df[(df["Data Negócio"].dt.year ==y) & (df["Data Negócio"].dt.month ==m)]["Código"].unique():
                venda_mes = df[(df["C/V"]=="V") & (df["Data Negócio"].dt.month ==m) & (df["Data Negócio"].dt.year ==y) & (df["Código"]==ticker)]['Valor Total (R$)'].sum()
                
                if venda_mes >= 20000:
                    for index in df[(df["C/V"] == 'V') & (df["Data Negócio"].dt.month == m) & (df["Data Negócio"].dt.year == y)& (df["Código"]==ticker)].index:
                        df.at[index, "DARF"] = df.loc[index]["Lucro da Venda"]* 0.15


                    valor = df[(df["C/V"] == 'V') & (df["Data Negócio"].dt.month == m) & (df["Data Negócio"].dt.year == y)& (df["Código"]==ticker)]["DARF"].sum()
                    print(f"O total imposto devido em relação as operações swing-trade no mes {m} do ano {y} escolhido é {round(valor,2)}R$")

    df_group = df[['Data Negócio', 'C/V', 'Código', 'Quantidade', 
'Valor Total (R$)','Custo de Operação', 'Lucro da Venda', 'Day/Swing', "DARF"]].groupby(['Data Negócio', "Código", "C/V"]).sum()
    
   
    return df_group[df_group['DARF']!=0]

In [45]:
def impostos(dataset,year ='todos',month='todos',day='todos',modalidade='todos'):

    if modalidade == "todos":
        df = dataset.copy()
    elif modalidade =='day':
        df = dataset[dataset['Day/Swing']=="Day"].copy()
    elif modalidade == 'swing':
        df = dataset[dataset['Day/Swing']=="Swing"].copy()
    else:
        print("Erro de modalidade")

    
    if year != 'todos' and month != 'todos' and day != 'todos':
        df_new = df[(df["Data Negócio"].dt.year == year) & (df["Data Negócio"].dt.month == month) & (df["Data Negócio"].dt.day == day)].copy()
    elif year != 'todos' and month != 'todos':
        df_new = df[(df["Data Negócio"].dt.year == year) & (df["Data Negócio"].dt.month == month)].copy()
    elif year != 'todos':
        df_new = df[(df["Data Negócio"].dt.year == year)].copy()
    else:
        df_new = df.copy()

    #creating a new column for DARF (tax)
    df_new["DARF"] = 0.

    #Calculating the tax for the Day-trades
    if modalidade == 'day':
        df_group = day_trade_imposto(df_new)
        df_group["Day/Swing"] = "Day"

    #Calculating the tax for the Swing-trades
    if modalidade == 'swing':
        df_group = swing_trade_imposto(df_new)
        df_group["Day/Swing"] = "Swing"

    #calculating the tax for both types
    if modalidade =='todos':
        df_group1 = day_trade_imposto(df_new)
        df_group1["Day/Swing"] = "Day"

        df_group2 = swing_trade_imposto(df_new)
        df_group2["Day/Swing"] = "Swing"

        df_group = pd.concat([df_group1,df_group2])

    return df_group


In [35]:
impostos(test, year = 2020, month=7, day=29, modalidade = 'day')

O total imposto devido em relação as operações day-trade no periodo escolhido é 778.29


Quantidade  Valor Total (R$)  Custo de Operação  \
Data Negócio Código C/V                                                    
2020-07-29   OIBR4  V            99             386.1          -0.144787   
             VVAR3  V            78            1586.6          -0.594975   

                         Lucro da Venda        DARF  
Data Negócio Código C/V                              
2020-07-29   OIBR4  V        730.772212  146.154442  
             VVAR3  V       3160.669025  632.133805

In [46]:
impostos(test)

O total imposto devido em relação as operações day-trade no periodo escolhido é 10.13
O total imposto devido em relação as operações swing-trade no mes 5 do ano 2020 escolhido é 24.35R$
O total imposto devido em relação as operações swing-trade no mes 7 do ano 2020 escolhido é 66.63R$


Quantidade  Valor Total (R$)  Custo de Operação  \
Data Negócio Código C/V                                                    
2020-07-27   GGBR4  V            10             174.3          -0.065363   
2020-07-29   OIBR4  V            99             386.1          -0.144787   
             VVAR3  V            78            1586.6          -0.594975   
2020-05-25   COGN3  V         10000           47200.0         -17.700000   
2020-07-28   VALE3  V           500           30150.0         -11.306250   

                         Lucro da Venda       DARF Day/Swing  
Data Negócio Código C/V                                       
2020-07-27   GGBR4  V         -1.825362  -0.365072       Day  
2020-07-29   OIBR4  V         41.138212   8.227642       Day  
             VVAR3  V         11.341025   2.268205       Day  
2020-05-25   COGN3  V        162.300000  24.345000     Swing  
2020-07-28   VALE3  V        444.193750  66.629062     Swing

In [44]:
impostos(test, year = 2020)

O total imposto devido em relação as operações day-trade no periodo escolhido é 10.13
O total imposto devido em relação as operações swing-trade no mes 5 do ano 2020 escolhido é 24.35R$
O total imposto devido em relação as operações swing-trade no mes 7 do ano 2020 escolhido é 66.63R$


Quantidade  Valor Total (R$)  Custo de Operação  \
Data Negócio Código C/V                                                    
2020-07-27   GGBR4  V            10             174.3          -0.065363   
2020-07-29   OIBR4  V            99             386.1          -0.144787   
             VVAR3  V            78            1586.6          -0.594975   
2020-05-25   COGN3  V         10000           47200.0         -17.700000   
2020-07-28   VALE3  V           500           30150.0         -11.306250   

                         Lucro da Venda       DARF  
Data Negócio Código C/V                             
2020-07-27   GGBR4  V         -1.825362  -0.365072  
2020-07-29   OIBR4  V         41.138212   8.227642  
             VVAR3  V         11.341025   2.268205  
2020-05-25   COGN3  V        162.300000  24.345000  
2020-07-28   VALE3  V        444.193750  66.629062

In [39]:
test.head(10)

,Data Negócio,C/V,Código,Quantidade,Preço (R$),Valor Total (R$),Day/Swing,Custo de Operação,Custo Médio,Lucro da Venda
0,2020-05-08,C,COGN3,10000,4.70,-47000.00,Swing,-15.275000,4.702,0.0
1,2020-05-08,C,TAEE4,100,8.90,-890.00,Swing,-0.289250,8.903,0.0
2,2020-05-13,C,IRDM11,5,100.42,-502.10,Swing,-0.163183,100.453,0.0
3,2020-05-13,C,WEGE3,3,38.00,-114.00,Swing,-0.037050,38.012,0.0
4,2020-05-20,C,BBAS3,35,27.10,-948.50,Swing,-0.308262,27.109,0.0
5,2020-05-25,V,COGN3,10000,4.72,47200.00,Swing,-17.700000,0.000,162.3
6,2020-05-27,C,ITSA4,14,9.09,-127.26,Swing,-0.041360,9.093,0.0
7,2020-05-27,C,ITSA4,46,9.09,-418.14,Swing,-0.135896,9.093,0.0
8,2020-06-03,C,ITUB4,9,26.23,-236.07,Swing,-0.076723,26.239,0.0
9,2020-06-03,C,ITUB4,30,26.23,-786.90,Swing,-0.255742,26.239,0.0


In [25]:
df[(df["Data Negócio"].dt.year ==2020) & (df["Data Negócio"].dt.month ==7)]["Código"].unique()

array(['LWSA3', 'IRBR3', 'OIBR4', 'VVAR3', 'WEGE3', 'GGBR4', 'VALE3',
       'ABEV3'], dtype=object)